## Setting up Lysoszyme to run standard MD smulation
This notebook will guide you through how to use FESetup for setting up a protein simulation and generate output for various different simulation engines. 


The notebook forms part of the CCPBio-Sim workshop **Tackling Alchemistry with FESetup and Sire SOMD** run on the 10th of April 2018 at the University of Bristol.

*Author: Anotnia Mey   
Email: antonia.mey@ed.ac.uk*

**Reading time of the document: xx mins**

In [ ]:
#JJ NOTES:
#    - The text after cell 6 is incomplete.
#    - Cell 10 gives this warning:
#        /opt/conda/lib/python3.5/site-packages/pandas/io/parsers.py:2227: FutureWarning: split() requires a non-empty pattern match.
#          yield pat.split(line.strip())
#        /opt/conda/lib/python3.5/site-packages/pandas/io/parsers.py:2229: FutureWarning: split() requires a non-empty pattern match.
#          yield pat.split(line.strip())
#    - Cell 10 also does not display the commands to play with the visualization
#    - I don´t understand this cell: "As for the simulation, we will run 50 steps of molecular dynamics." 
#      it seems to run 10 cycles? Or is this the weird SOMD thing?
#    - Cell 26 does not seem to pause.
#    - Cell 30 returns an error.
#    - Not sure is Benzofurane or Benzofuran, but you should pick one (currently is Benzofurane in the notebook and the 
#     folder is benzofuran)

### Imports

In [1]:
%pylab inline
import nglview as nv
import BioSimSpace as BSS

Populating the interactive namespace from numpy and matplotlib


## Setting up Lysozyme 
FESetup has a nice feature, where it will produce compatible output for different MD engines. It currently support *NAMD*, *AMBER*, *Gromacs* and *DLPOLY*, provided the executables for each of the simulation packages are actually installed. In the following we will look at how to set up lysozyme. The first part of this tutorial is found in the directory Task01. 

In [2]:
cd Task01

/home/workshops/fesetup_workshop/02_Lysozyme_and_advanced_FESetup_topics/Task01


Let's start by loading the molecule.

In [3]:
view = BSS.viewMolecules('proteins/181L/protein.pdb')
view

Reading molecules from '['proteins/181L/protein.pdb']'
Rendering the molecules...


NGLWidget()

Tab(children=(Box(children=(Box(children=(Box(children=(Label(value='step'), IntSlider(value=1, min=-100)), layout=Layout(display='flex', flex_flow='row', justify_content='space-between')), Box(children=(Label(value='delay'), IntSlider(value=100, max=1000, min=10)), layout=Layout(display='flex', flex_flow='row', justify_content='space-between')), Box(children=(Label(value='background'), ColorPicker(value='white')), layout=Layout(display='flex', flex_flow='row', justify_content='space-between')), Box(children=(Label(value='camera'), Dropdown(options=('perspective', 'orthographic'), value='perspective')), layout=Layout(display='flex', flex_flow='row', justify_content='space-between')), Box(children=(Label(value=''), HBox(children=(ToggleButton(value=False, description='Smoothing', tooltip='smoothing trajectory'), Button(description=' Center', icon='bullseye', style=ButtonStyle()), Button(description=' Screenshot', icon='camera', style=ButtonStyle()), Button(description='qtconsole', style=ButtonStyle(), tooltip='pop up qtconsole')), layout=Layout(flex='1 1 auto', width='auto'))), layout=Layout(display='flex', flex_flow='row', justify_content='space-between'))), layout=Layout(align_items='stretch', display='flex', flex_flow='column', width='100%')),)), Box(), Box(), Box(), Box(), Box()), layout=Layout(align_items='stretch', align_self='center'), _titles={'1': 'Representation', '3': 'Theme', '2': 'Preference', '0': 'General', '5': 'Help', '4': 'Extra'})

As for the ethane methanol simulations we have an input file for FESetup. This time it has a new section named [protein]. This contains all the relevant options for setting up a protein simulation. Let's have a look at this very basic setup file. 

In [4]:
!head -n 30 setup.in

# T4-lysozyme: setup

logfile = T4-lysozyme_molsetup.log
forcefield = amber, ff14SB, tip3p, cm

mdengine = amber, sander

# prepare the biomolecules (could also be DNA for example)
[protein]
basedir = proteins
molecules = 181L 
neutralize = True 

box.type = rectangular
box.length = 12.0
align_axes = True

min.nsteps = 200
min.ncyc = 100



We want to put the protein 181L into a box of length 12 Å, add some solvent, neutralize the box and run a set of minimisation steps. Executing this will generate the ouptut file `_proteins` which will contain the minimised protein structure. 

In [5]:
!FESetup setup.in


=== FESetup release 1.2.1, SUI version: 0.8.3 ===

Please cite: HH Loeffler, J Michel, C Woods, J Chem Inf Mod, 55, 2485
             DOI 10.1021/acs.jcim.5b00368
For more information please visit http://www.ccpbiosim.ac.uk/fesetup/

Making biomolecule 181L...

=== All molecules built successfully ===



In [6]:
ls _proteins/181L

leap.log     min00001.info  protein.pdb     solvated.rst7  vacuum.rst7
min00001.en  min00001.out   solvated.parm7  vacuum.parm7
min00001.in  min00001.rst7  solvated.pdb    vacuum.pdb


This directory now contains your solvated protein ready for an equilibration or any other type of simulation run. This is the simplest way of preparing a protein simulation. Of course you can again use 

### Setup with Gromacs
We can use the same setup to run with gromacs by simply changing the line specifying the MD engine:   
`
mdengine = gromacs, mdrun
`   
If Gromacs is not installed the setup will fail. This is currently the case on this server, but we can still run an example file until we get the error. This time we use `182L` for the setup. 

In [7]:
!FESetup setup_gromacs.in


=== FESetup release 1.2.1, SUI version: 0.8.3 ===

Please cite: HH Loeffler, J Michel, C Woods, J Chem Inf Mod, 55, 2485
             DOI 10.1021/acs.jcim.5b00368
For more information please visit http://www.ccpbiosim.ac.uk/fesetup/

Making biomolecule 182L...
ERROR: 182L failed: GMXHOME does not have a bin directory
ERROR: The following proteins have failed:
 182L


While we have recieed an error, because we don't have Gromacs installed and can't run the minimisation, we can have a look in the `_proteins/182L` directory. You will find it contains a `solvated.gro` and `solvated.top` file, which are the Gromacs coordinate and topology files respectively and can be used to run your simulation elsewhere.  

In [8]:
ls _proteins/182L

leap.log     solvated.gro    solvated.pdb   solvated.top  vacuum.pdb
protein.pdb  solvated.parm7  solvated.rst7  vacuum.parm7  vacuum.rst7


**Task: What would you have to change in `setup.in` to prepare files compatible with NAMD?**

## Setting up Lysozyme with a ligand 
Setting up a single protein isn't necessarily such a complcated task. But setting up a protein and a ligand together might require more effort. We will often lack compatible parameters and partial charges and generating them by hand can be cumbersome. FESetup can help with this. Let's look at Task02 working with both protein and ligand. 

### Benzene as a ligand of Lysozyme

In [9]:
cd ../Task02

/home/workshops/fesetup_workshop/02_Lysozyme_and_advanced_FESetup_topics/Task02


You now have a proteins and ligands directory. The proteins directory contains again the structure of 181L and the ligands directory that of benzene.    

**Task: visualise both the protein and the ligand to make sure that the ligand has the right coordinates**

In [12]:
#change appropriate code below
import mdtraj as md
protein = md.load('proteins/181L/protein.pdb')
ligand = md.load('ligands/benzene/ligand.mol2')
from nglview import NGLWidget
view = NGLWidget()
view.add_trajectory(protein)
view.add_trajectory(ligand)
view

/opt/conda/lib/python3.5/site-packages/pandas/io/parsers.py:2227: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/opt/conda/lib/python3.5/site-packages/pandas/io/parsers.py:2229: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())


NGLWidget()

Now we want to actually greate a complex of the ligand and protein and solvate it. For this there is another `setup.in` file prepared. Let's look at this file a bit more closely:

In [13]:
!cat setup.in

# T4-lysozyme: 

logfile = T4-lysozyme_molsetup.log
forcefield = amber, ff14SB, tip3p, cm

mdengine = amber, sander

#setus up ligand benzene in a solvated box and in vacuum
[ligand]
basedir = ligands
file.name = ligand.mol2
molecules = benzene

box.type = rectangular
box.length = 12.0
neutralize = yes
min.nsteps = 100

# prepare the biomolecules (could also be DNA for example)
[protein]
basedir = proteins
molecules = 181L 
neutralize = True


# create the complex form the ligands and proteins above, solvate and minimise
# the complexes
[complex]
# explicit enumeration of pairs, otherwise all-with-all creation even
# when there is no [complex] section
pairs = 181L : benzene

box.type = rectangular
box.length = 12.0
align_axes = True

min.nsteps = 200
min.ncyc = 100



We again have a [ligand] section which defines the molecule and also makes sure that benzene will be set up in a solvation box. Then again a [protein] section,and in addition there is now a [complex] section, which defines the fact that we want to create a complex of 181L and benzene in a box and solvate and minimise it. 

In [14]:
# Insert the code here to run the setup. 
!FESetup setup.in


=== FESetup release 1.2.1, SUI version: 0.8.3 ===

Please cite: HH Loeffler, J Michel, C Woods, J Chem Inf Mod, 55, 2485
             DOI 10.1021/acs.jcim.5b00368
For more information please visit http://www.ccpbiosim.ac.uk/fesetup/

Making biomolecule 181L...
Making ligand benzene...
Making complex from 181L and benzene...

=== All molecules built successfully ===



In [15]:
ls _complexes/181L:benzene/

gaff.mol2      ligand.mol2    min00001.out     not_first.rst7  solvated.rst7
leap.log       min00001.en    min00001.rst7    protein.pdb     vacuum.parm7
ligand.ac      min00001.in    not_first.parm7  solvated.parm7  vacuum.pdb
ligand.frcmod  min00001.info  not_first.pdb    solvated.pdb    vacuum.rst7


Now you should be able to located a directory called `_complex` that was generated by FESetup. It should contain the complex in vacuum and in solvent. 

In [16]:
# Insert code to visualise the complex in vacuum
view = BSS.viewMolecules('_complexes/181L:benzene/vacuum.pdb')


Reading molecules from '['_complexes/181L:benzene/vacuum.pdb']'
Rendering the molecules...


NGLWidget()

Tab(children=(Box(children=(Box(children=(Box(children=(Label(value='step'), IntSlider(value=1, min=-100)), layout=Layout(display='flex', flex_flow='row', justify_content='space-between')), Box(children=(Label(value='delay'), IntSlider(value=100, max=1000, min=10)), layout=Layout(display='flex', flex_flow='row', justify_content='space-between')), Box(children=(Label(value='background'), ColorPicker(value='white')), layout=Layout(display='flex', flex_flow='row', justify_content='space-between')), Box(children=(Label(value='camera'), Dropdown(options=('perspective', 'orthographic'), value='perspective')), layout=Layout(display='flex', flex_flow='row', justify_content='space-between')), Box(children=(Label(value=''), HBox(children=(ToggleButton(value=False, description='Smoothing', tooltip='smoothing trajectory'), Button(description=' Center', icon='bullseye', style=ButtonStyle()), Button(description=' Screenshot', icon='camera', style=ButtonStyle()), Button(description='qtconsole', style=ButtonStyle(), tooltip='pop up qtconsole')), layout=Layout(flex='1 1 auto', width='auto'))), layout=Layout(display='flex', flex_flow='row', justify_content='space-between'))), layout=Layout(align_items='stretch', display='flex', flex_flow='column', width='100%')),)), Box(), Box(), Box(), Box(), Box()), layout=Layout(align_items='stretch', align_self='center'), _titles={'1': 'Representation', '3': 'Theme', '2': 'Preference', '0': 'General', '5': 'Help', '4': 'Extra'})

**Tip: Play around with the representations in order to be able to view both the ligand and the protein**

## Butylbenzene as a ligand of Lysosyme
Often you are in a position where you might want to run a simulation with a ligand where you only have a smiles string or sdf file. So how can we go from a smiles file to a solvated complex? One easy way to do this is using babel. 
Take a look at `ligands/ortho-xylene/ligand.smi`. This file contains the following smiles string: `CC1=C(C)C=CC=C1`. What we need to do it generate a 3D structure.   

The next cell uses babel to convert the input smiles string to a 3D structure of format mol2. 

In [17]:
!obabel -ismi ligands/ortho-xylene/ligand.smi  -opdb ligands/ortho-xylene/ligand.pdb --gen3D -h  > ligands/ortho-xylene/ligand.pdb

1 molecule converted


Double check that the ligand molecule has been generated!

Next there is already a perapred input file for ortho-xylene! Let's just run the setup!

In [18]:
!FESetup setup_xylene.in


=== FESetup release 1.2.1, SUI version: 0.8.3 ===

Please cite: HH Loeffler, J Michel, C Woods, J Chem Inf Mod, 55, 2485
             DOI 10.1021/acs.jcim.5b00368
For more information please visit http://www.ccpbiosim.ac.uk/fesetup/

Making biomolecule 181L...
Making ligand ortho-xylene...
Making complex from 181L and ortho-xylene...

=== All molecules built successfully ===



Let's have a look at the generated complex for ortho-xylene. There seems to be a problem with the way the ligand is positioned!

In [19]:
view = BSS.viewMolecules('_complexes/181L:ortho-xylene/vacuum.pdb')

Reading molecules from '['_complexes/181L:ortho-xylene/vacuum.pdb']'
Rendering the molecules...


NGLWidget()

Tab(children=(Box(children=(Box(children=(Box(children=(Label(value='step'), IntSlider(value=1, min=-100)), layout=Layout(display='flex', flex_flow='row', justify_content='space-between')), Box(children=(Label(value='delay'), IntSlider(value=100, max=1000, min=10)), layout=Layout(display='flex', flex_flow='row', justify_content='space-between')), Box(children=(Label(value='background'), ColorPicker(value='white')), layout=Layout(display='flex', flex_flow='row', justify_content='space-between')), Box(children=(Label(value='camera'), Dropdown(options=('perspective', 'orthographic'), value='perspective')), layout=Layout(display='flex', flex_flow='row', justify_content='space-between')), Box(children=(Label(value=''), HBox(children=(ToggleButton(value=False, description='Smoothing', tooltip='smoothing trajectory'), Button(description=' Center', icon='bullseye', style=ButtonStyle()), Button(description=' Screenshot', icon='camera', style=ButtonStyle()), Button(description='qtconsole', style=ButtonStyle(), tooltip='pop up qtconsole')), layout=Layout(flex='1 1 auto', width='auto'))), layout=Layout(display='flex', flex_flow='row', justify_content='space-between'))), layout=Layout(align_items='stretch', display='flex', flex_flow='column', width='100%')),)), Box(), Box(), Box(), Box(), Box()), layout=Layout(align_items='stretch', align_self='center'), _titles={'1': 'Representation', '3': 'Theme', '2': 'Preference', '0': 'General', '5': 'Help', '4': 'Extra'})

**Questions: Can you think of ways to align the ligand to the correct binding site of the protein? Write down a few ideas in the next cell:**

**Advanced task**: *Come back to this if you still have time at the end of the tutorial and implement a way to put xylene in an appropirate position in the binding site. Alternative the BioSimSpace Workshop will cover an easy way of how to do this.*

## Running MD of Lysozyme with benzene bound
We can now use the generated complex files to run standard MD simulations. This could be done with different simulation software. Here we will be using Sire's SOMD. In the directory `Task03/simulation` an input file has already been perpared. SOMD takes standard Amber input files as input. Have a look at what has been provided. 

In [22]:
cd ../Task03

/home/workshops/fesetup_workshop/02_Lysozyme_and_advanced_FESetup_topics/Task03


In [23]:
pwd

'/home/workshops/fesetup_workshop/02_Lysozyme_and_advanced_FESetup_topics/Task03'

We will take a closer look at the config file. `solvated.rst7` and `solvated.parm7` are the coordinate file and topology file respectively. The complex was prepared with an NVT heating and an NPT equilibration which we skipped during task two to avoid very long waiting times for molecular struactures to be generated. Prepping these kind of files on a cluster or powerful computer should only take a few minutes though. For convenience please use the files we provided and are ready for a production simulation. 

In [24]:
!cat sim.cfg

# Length of simulation and platform
nmoves = 10 
ncycles = 10 # 10 moves x 10 cycles x 2fs = 200fs 
platform = CPU
# Input files 
topfile = solvated.parm7 
crdfile = solvated.rst7 

#Potential energy function parameters
cutoff type = cutoffperiodic 
cutoff distance = 10 * angstrom 
reaction field dielectric = 78.3 
# Beyond cutoff is uniform with this dielectric constant
#shift delta = 2.0 
# MD integration parameters
timestep = 2 * femtosecond 
constraint = hbonds 
integrator type = leapfrogverlet 
center of mass frequency = 10
# Temperature/Pressure control
temperature = 298.15 * kelvin 
pressure = 1 * atm 
andersen = True 
andersen frequency = 10 
barostat = True 
barostat frequency = 25
# Equilibration protocol
minimise = False # False if already minimised.
# Output control
save coordinates = True 
buffered coordinates frequency = 0 #save every frame 
# 10 moves x 10 cycles = 100.
# Snapshots =100/10 = 10 snapshots.


The above runs 200 fs of molecular dynamics for the protein benzene complex using the NPT ensemble. Most directives of the config files should be quite self explanatory. the buffered coordinates frequency set to 0 means that every single frame is saved which is fine for a simulation of 100 frames. The executable for running straight up MD simulations is `somd` and comes with Sire. Usually it is adviced to have the platform set to `CUDA` or `OpenCL` as `somd` uses `OpenMM` to accelrate integration on a GPU. This cloud server does not have any GPUs available, therefore we must default to the CPU platform. 

In [25]:
!somd -C sim.cfg

Starting somd: number of threads equals 8

Loading configuration information from file sim.cfg
Using parameters:
# 10 moves x 10 cycles == 100.0
# Snapshots == 100/10 = 10 snapshots.
#shift delta == 2.0
andersen == True
andersen frequency == 10
barostat == True
barostat frequency == 25
buffered coordinates frequency == 0
center of mass frequency == 10
constraint == hbonds
crdfile == solvated.rst7
cutoff distance == 10 angstrom
cutoff type == cutoffperiodic
integrator type == leapfrogverlet
minimise == False
ncycles == 10
nmoves == 10
platform == CPU
pressure == 1 atm
reaction field dielectric == 78.3
save coordinates == True
temperature == 298.15 t
timestep == 0.002 ps
topfile == solvated.parm7

### Running Molecular Dynamics simulation on jupyter-597366bb-2d3d6b-2d43a8-2dbb46-2d7116f6e34cd3 ###
###================Setting up calculation=====================###
New run. Loading input and creating restart
Creating the system...
Creating force fields... 
Setting up the simulation with ran

As for the simulation, we will run 50 steps of molecular dynamics. 

In [26]:
protein = md.load('traj000000001.dcd', top='solvated.pdb')
view = NGLWidget()
view.add_trajectory(protein)
view

NGLWidget(count=10)

In [27]:
print (protein)

<mdtraj.Trajectory with 10 frames, 29441 atoms, 9105 residues, and unitcells>


**Advanced Task: Can you compute and plot the RMSD of the benzene ligands over the saved coordinates?**

In [28]:
#Insert code for advance task here



## Setting up ligand perturbations with a protein
If you want to compute relative binding free energies of a set of ligands to a protein using an alchemical approach you will need to generate a pertubration map. In Task04 you will generate a set of ligands that are all similar and you need to define a perturbation map for them. Just think of what was discussed in the lecture in terms of perturbation maps. You want:
- many cycles, to make sure you can calculate robustness
- easy perturbations, avoid breaking large rings, if needed add intermediates
- always consider forward and backward perturbations. 

In [29]:
cd ../Task04

/home/workshops/fesetup_workshop/02_Lysozyme_and_advanced_FESetup_topics/Task04


In [30]:
ls images/

ls: cannot access 'images/': No such file or directory


We have the following ligands:
- indene,
- benzene,
- benzofurane
- indole
- o-xylene
- p-xylene
Have a look at their structures:
![foo](images/structures.png)

Tasks:
1. Can you change molsetup.in in such a way that all molecules are setup with FESetup and a complex is created the 181L structure without running a proper equilibration to avoid long execution times. 
2. can you suggest a set of perturbations and add them to morph.in to generate a perturbation map that is sensible in order to assess relative binding free energies of the above 6 molecules. 
3. Make sure you run molsetup.in and morph.in setup and look at the generated `_perturbations` directory which can then be used for running somd alchemical free energy calculations

In [33]:
# adjust molesetup.in before running this cell
!FESetup molsetup.in


=== FESetup release 1.2.1, SUI version: 0.8.3 ===

Please cite: HH Loeffler, J Michel, C Woods, J Chem Inf Mod, 55, 2485
             DOI 10.1021/acs.jcim.5b00368
For more information please visit http://www.ccpbiosim.ac.uk/fesetup/

Making biomolecule 181L...
Making ligand benzofurane...
Making complex from 181L and benzofurane...

=== All molecules built successfully ===



In [35]:
# adjust morph.in before running this cell
!FESetup morph.in


=== FESetup release 1.2.1, SUI version: 0.8.3 ===

Please cite: HH Loeffler, J Michel, C Woods, J Chem Inf Mod, 55, 2485
             DOI 10.1021/acs.jcim.5b00368
For more information please visit http://www.ccpbiosim.ac.uk/fesetup/

Making biomolecule 181L...
Morphs will be generated for Sire
Morphing benzene to o-xylene...
Morphing o-xylene to benzene...
Morphing benzene to p-xylene...
Morphing p-xylene to benzene...
Morphing p-xylene to o-xylene...
Morphing o-xylene to p-xylene...
Morphing benzene to indene...
Morphing indene to benzene...
Morphing indene to indole...
Morphing indole to indene...
Morphing indene to benzofurane...
Morphing benzofurane to indene...
Morphing benzofurane to indole...
Morphing indole to benzofurane...
Creating complex 181L:benzene with ligand morph benzene~o-xylene...
Creating complex 181L:o-xylene with ligand morph o-xylene~benzene...
Creating complex 181L:benzene with ligand morph benzene~p-xylene...
Creating complex 181L:p-xylene with ligand morph p-

In [ ]:
# Add code to visualise some of your generated output. 




## Advanced topics: explicit mapping 
Sometimes the proposed atom mapping for the morphing is not quite what you might want to do. For this purpose FESetup has a built in feature that lets you explicitly map the atoms. This may become imporant with chiral molecules or symmetric molecules or when you think a mapping makes more sense than what is proposed by the automatic algorithm. Think of this for example:
![foo](images/mapping.png)
![foo](images/mapping2.png)
Let's look at Task05 and see how we can achieve and explicit atom mapping. 

In [36]:
cd ../Task05

/home/workshops/fesetup_workshop/02_Lysozyme_and_advanced_FESetup_topics/Task05


You can explicity state the mapping in the `setup.in` file in this way:   
```
[ligand]
basedir = ligands
morph_pairs = benzene > benzofuran /1=3/2=2  # indices start from 1
```

Or you can use a `.map` file that sits in your ligands base directory to do the mapping. 

The content of the equivalent map file would look like this:   
```
# example mapping file benzene~benzofuran.map in the basedir ligands/
# explicitly map the following atom indexes onto each other
1 3 # this mapping and...
2 2 # ...this one will fix the orientation of benzofuran in space

```

**Tasks: **
1. Run FESetup using an explicit mapping or 1=3 and 2=2 in the setup.in file.
2. Run FESetup using a map file to do the same mapping attaching to 3 and 4 of the benzene C atoms

Please modify the existing file `setup.in` and `ligands/benzene~benzofurane.map` files to complete the task!

In [ ]:
#The changes to the files will have to be done in the terminal, but feel free to add the code to execute FESetup here




## A little quizz for the end
Please have a quick look at this quizz to see what you have learned from this notebook:
https://goo.gl/forms/hSsPTYxnWvKjWj2B3